# Análisis de Opiniones sobre Películas

Cargando librerias...

In [1]:
# coding=UTF-8
import urllib
import pandas as pd
import numpy as np
import re, time
import random
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import WordNetLemmatizer, word_tokenize, data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
#comentar a siguiente linea si se tienen instalado en el path por defecto
data.path.append('nltk_data')

### **(a) Dataframe**
Se cargan los datos a analizar para crear el dataframe que se utilizará en el desarrollo del problema de análisis de opiniones de películas. Se utilizará un set de entrenamiento y un set de prueba.  Ambos documentos presentan dos columnas (sin contar el índice). La primera contiene el **Sentimiento** que toma valor de **+1** cuando es positivo y **-1** cuando es negativo. La segunda columna corresponde al texto u opinión escrita por una persona respecto a una película. El siguiente es un extracto de los datos

|Sentiment|Text|
|:------|------:-----|
|-1|everything's serious , poetic , earnest and --...|
|-1|narratively , trouble every day is a plodding ...|
|+1|a truly wonderful tale combined with stunning ...|

In [2]:
# cargar datos
train_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.train"
test_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.dev"
train_data_f = urllib.urlretrieve(train_data_url, "train_data.csv")
test_data_f = urllib.urlretrieve(test_data_url, "test_data.csv")
ftr = open("train_data.csv", "r")
fts = open("test_data.csv", "r")

# separar lineas en dos columnas: sentimiento y texto u opinion
rows = [line.split(" ",1) for line in ftr.readlines()]
train_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
train_df['Sentiment'] = pd.to_numeric(train_df['Sentiment'])
rows = [line.split(" ",1) for line in fts.readlines()]
test_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
test_df['Sentiment'] = pd.to_numeric(test_df['Sentiment'])

Una vez armados los dataframes, la cantidad de entradas que tiene cada uno es de 3554 filas y 2 columnas.

In [3]:
print "Cantidad de registros train set ",train_df.shape
print "Cantidad de registros test set ",test_df.shape

Cantidad de registros train set  (3554, 2)
Cantidad de registros test set  (3554, 2)


### **(b) Stemming:**
La siguiente función *word_extractor* devuelve una lista de las palabras contenidas en un determinad trozo de texto. Incluye las operaciones de lower-casing y stemming. Stemming es el proceso de reducir una palabra a su raiz (stem). En este caso se utilizará el algoritmo de Porter. Corta las palabras esperando encontrar un sufijo, incluyendo eliminación de afijos

In [4]:
def word_extractor(text,sw=True,stmmng=True):
    porterstemmer = PorterStemmer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    if stmmng:
        wordtokens = [ porterstemmer.stem(word.lower()) for word in word_tokenize(text.decode('utf-8', 'ignore')) ]
    else:
        wordtokens = [ word.lower() for word in word_tokenize(text.decode('utf-8', 'ignore')) ]
    for word in wordtokens:
        if sw:
            if word not in commonwords:
                words+=" "+word
        else:
            words+=" "+word
    return words

print "Stemming"
print word_extractor("I love to eat cake denied")
print word_extractor("I love eating cake")
print word_extractor("I loved eating the cake")
print word_extractor("I do not love eating cake")
print word_extractor("I don't love eating cake")
print word_extractor("You died last WeEk. It was sensational")
print word_extractor("owned cats, stemming cakes, factionally running") 

print "\nSin Stemming"
print word_extractor("I love to eat cake denied",stmmng=False)
print word_extractor("I love eating cake",stmmng=False)
print word_extractor("I loved eating the cake",stmmng=False)
print word_extractor("I do not love eating cake",stmmng=False)
print word_extractor("I don't love eating cake",stmmng=False)
print word_extractor("You died last WeEk. It was sensational",stmmng=False)
print word_extractor("owned cats, stemming cakes, factionally running",stmmng=False) 

Stemming
 love eat cake deni
 love eat cake
 love eat cake
 love eat cake
 n't love eat cake
 die last week . wa sensat
 cat , stem cake , faction run

Sin Stemming
 love eat cake denied
 love eating cake
 loved eating cake
 love eating cake
 n't love eating cake
 died last week . sensational
 owned cats , stemming cakes , factionally running


Se puede ver como al aplicar stemming se cortan las palabras elimindo afijos y sufijos, además de los stop words. Sin stemming se eliminan las stop words pero las palabras quedan completas, no se cortan. En este caso la cantidad de palabras en el vocabulario aumenta pues, para una misma raiz de palabra, habrán muchas otras, en especial con las conjugaciones de verbos. Por ejemplo, running aplicando stemming se corta a run pero cuando no se aplica stemming queda solamente running, por lo tanto al tener running run quedarán como palabras separadas del vocabulario.

### **(c) Lematizador: **
Lematización es el proceso de convertir una palabra a su lema. Se eliminan formas en plural, femenino, conjugaciones. La palabra que se obtiene es una palabra del vocabulario presente en un diccionario. Por ejemplo el lema de nadé sería nadar.

In [5]:
def word_extractor2(text,sw):
    wordlemmatizer = WordNetLemmatizer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ wordlemmatizer.lemmatize(word.lower()) for word in word_tokenize(text.decode('utf-8','ignore')) ]
    for word in wordtokens:
        if sw:
            if word not in commonwords:
                words+=" "+word
        else:
            words+=" "+word
    return words

print "\nLematizador"
print word_extractor2("I love to eat cake denied",True)
print word_extractor2("I love eating cake",True)
print word_extractor2("I loved eating the cake",True)
print word_extractor2("I do not love eating cake",True)
print word_extractor2("I don't love eating cake",True)
print word_extractor2("You died last WeEk. It was sensational",True)
print word_extractor2("owned cats, stemming cakes, tigers run running",True)


Lematizador
 love eat cake denied
 love eating cake
 loved eating cake
 love eating cake
 n't love eating cake
 died last week . wa sensational
 owned cat , stemming cake , tiger run running


Algunas palabras quedan similares a las procesadas por stemming pero a diferencia ahora son cortadas.

### **(d) Representación vectorial: **
En el siguiente paso se construyó una función para generar una representación vectorial de los textos, aplicando lematización o stemming, con el eliminación y no limpieza de stop words. 

In [6]:
def vector_rep(method,sw=True):
    if method == "stemming":
        if sw:
            texts_train    = [word_extractor(text) for text in train_df.Text]
            texts_test     = [word_extractor(text) for text in test_df.Text] 
        else:
            texts_train    = [word_extractor(text,sw=False) for text in train_df.Text]
            texts_test     = [word_extractor(text,sw=False) for text in test_df.Text]  
    elif method == "lemmatisation":
        if sw:
            texts_train    = [word_extractor2(text,True) for text in train_df.Text]
            texts_test     = [word_extractor2(text,True) for text in test_df.Text]
        else:
            texts_train    = [word_extractor2(text,False) for text in train_df.Text]
            texts_test     = [word_extractor2(text,False) for text in test_df.Text]

    vectorizer     = CountVectorizer(ngram_range=(1, 1), binary='False')
    vectorizer.fit(np.asarray(texts_train))
    features_train = vectorizer.transform(texts_train)
    features_test  = vectorizer.transform(texts_test)
    labels_train   = np.asarray((train_df.Sentiment.astype(float)+1)/2.0)
    labels_test    = np.asarray((test_df.Sentiment.astype(float)+1)/2.0)
    vocab          = vectorizer.get_feature_names()
    dist_train     = list(np.array(features_train.sum(axis=0)).reshape(-1,))
    return features_train, features_test, labels_train, labels_test, vocab, dist_train 

Obtenida la representación vectorial del training set y test set se puede ver cuales son las diez palabras más frecuentes dentro de las opiniones.

In [7]:
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep("lemmatisation")
print "Set de entrenamiento: ",features_train.shape
print "Set de pruebas: ", features_test.shape
train_tags = []
test_tags = []
dist_test = list(np.array(features_test.sum(axis=0)).reshape(-1,))

for tag, count in zip(vocab, dist_train):
    train_tags.append([count, tag])
for tag, count in zip(vocab, dist_test):
    test_tags.append([count, tag])

train_tags.sort(reverse=True)
test_tags.sort(reverse=True)
print "\n----\nPalabras más frecuentes en set de entrenamiento\nRepeticiones\tPalabra"
for i in range(10):
    print "%d\t\t%s" % (train_tags[i][0], train_tags[i][1])
print "\nPalabras más frecuentes en set de prueba\nRepeticiones\tPalabra"
for i in range(10):
    print "%d\t\t%s" % (test_tags[i][0], test_tags[i][1])

Set de entrenamiento:  (3554, 9663)
Set de pruebas:  (3554, 9663)

----
Palabras más frecuentes en set de entrenamiento
Repeticiones	Palabra
566		film
481		movie
246		one
245		like
224		ha
183		make
176		story
163		character
145		comedy
143		time

Palabras más frecuentes en set de prueba
Repeticiones	Palabra
558		film
540		movie
250		one
238		ha
230		like
197		story
175		character
165		time
161		make
134		comedy


En ambos casos, train set y test set, están compuestos de 9663 palabras. En ambos casos se repiten las palabras, estando film y movie obviamente arriba del ranking, pues se está hablando de películas. El resto de las palabras tienen diferente orden y esto depende de como fueron analizadas por stemming y lematizador. Los métodos no lograban llegar, en todos los casos, a la misma raiz de las palabras. Así, la distribución de ellas cambia.

### **(e) Función de desempeño**:

Se procedió a crear la función de evaluación de desempeño para los clasificadores. Las métricas que utiliza la función *classification_report* de la librería sklearn son tres:

1. **Precision**: es el ratio de verdaderos positivos divididos por la cantidad de casos positivos (falsos positivos +verdaderos positivos). Es la habilidad del clasificador de no etiquetar una muestra como positiva cuando esta es negativa.
2. **Recall**: Es el ratio entre verdaderos positivos y la cantidad de verdaderos esperados. Es la habilidad del clasificador de encontrar todos las muestras positivas. 
3. **f1-score**: Es la medida de precisión que tiene un test. Tiene su mejor caso cuando alcanza uno y su peor caso cuando alcanza cero. Está relacionada con precision y recall, calculándose de la siguiente manera: 
\begin{equation*}
F1 = 2 * \frac{precision * recall}{precision + recall}
\end{equation*}


In [8]:
def score_the_model(model,x,y,xt,yt,text):
    acc_tr = model.score(x,y)
    acc_test = model.score(xt[:-1],yt[:-1])
    print "Training Accuracy %s: %f"%(text,acc_tr)
    print "Test Accuracy %s: %f"%(text,acc_test)
    print "Detailed Analysis Testing Results ..."
    print (classification_report(yt, model.predict(xt), target_names=['+','-']))
    #computar precisión recall fscore y soporte para cada clase
    precision, recall, fscore, support = precision_recall_fscore_support(yt, model.predict(xt))

    if text == 'BernoulliNB':
        BNB = []
        BNB.append((acc_tr,acc_test,np.mean(precision),np.mean(recall),np.mean(fscore)))
    elif text == 'MULTINOMIAL':
        multinomial = []
        multinomial.append((acc_tr,acc_test,np.mean(precision),np.mean(recall),np.mean(fscore)))
    elif text == 'LOGISTIC':
        logistic = []
        logistic.append((acc_tr,acc_test,np.mean(precision),np.mean(recall),np.mean(fscore)))
    elif text == 'SVM':
        svm = []
        svm.append((acc_tr,acc_test,np.mean(precision),np.mean(recall),np.mean(fscore)))

### **(f) Clasificador Bayesiano Ingenuo (binario): **
Se corrió el clasificador Naive Bayes para stemming y lematizador, dejando y eliminando stop words para ver su comportamiento. Las features son transformadas a 1 y 0 para su funcionamiento.

In [9]:
def do_NAIVE_BAYES(x,y,xt,yt):
    model = BernoulliNB()
    model = model.fit(x, y)
    score_the_model(model,x,y,xt,yt,"BernoulliNB")
    return model

BNB = []
print "Naive Bayes - BernoulliNB"
print "Lematizador con Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('lemmatisation')
model=do_NAIVE_BAYES(features_train,labels_train,features_test,labels_test)
test_pred = model.predict_proba(features_test)

print "Lematizador sin Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('lemmatisation',sw=False)
model=do_NAIVE_BAYES(features_train,labels_train,features_test,labels_test)

print "Stemming con Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('stemming')
model=do_NAIVE_BAYES(features_train,labels_train,features_test,labels_test)

print "Stemming con Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('stemming',sw=False)
model=do_NAIVE_BAYES(features_train,labels_train,features_test,labels_test)

print "Conjunto aleatorio de prueba"
spl = random.sample(xrange(len(test_pred)), 5)
for text, sentiment in zip(test_df.Text[spl], test_pred[spl]):
    print sentiment, text


Naive Bayes - BernoulliNB
Lematizador con Stopwords
Training Accuracy BernoulliNB: 0.958638
Test Accuracy BernoulliNB: 0.738531
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.75      0.73      0.74      1803
          -       0.73      0.75      0.74      1751

avg / total       0.74      0.74      0.74      3554

Lematizador sin Stopwords
Training Accuracy BernoulliNB: 0.955262
Test Accuracy BernoulliNB: 0.748663
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.74      0.75      1803
          -       0.74      0.76      0.75      1751

avg / total       0.75      0.75      0.75      3554

Stemming con Stopwords
Training Accuracy BernoulliNB: 0.942881
Test Accuracy BernoulliNB: 0.747819
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.74      0.75      1803
          -       0.

Se puede ver como BernoulliNB tiene un alto valor de accuracy en el training set pero para el test set este disminuye en una considerable proporción. Esto quiere decir que el modelo está sobreajustado respecto a los datos. Como está sobreajustado la calidad de predicción no es muy buena, información que se puede corroborrar con los valores de precision y recall.

Stemming tiene resultados similares a los de lematización, siendo este último el mejor cuando no se eliminan los stop words. Para seleccionar el mejor, dado que los resultados son cercanos, se puede ocupar un criterio de tiempo de ejecución.

Para todos los casos la accuracy de entrenamiento es mejor que la de testing. Una conclusión, sin ver los datos, es que pueden existir palabras que no estén presentes en el test de entrenamiento. En este caso habría problema clasificándolas.

### **(g) Clasificador bayesiano ingenuo Multinomial:**
Se corrió el clasificador Multinomial Naive Bayes para stemming y lematizador, dejando y eliminando stop words para ver su comportamiento. Las features son transformadas a 1 y 0 para su funcionamiento.

In [10]:
def do_MULTINOMIAL(x,y,xt,yt):
    model = MultinomialNB()
    model = model.fit(x, y)
    score_the_model(model,x,y,xt,yt,"MULTINOMIAL")
    return model

multinomial = []
print "BI Multinomial"
print "Lematizador con Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('lemmatisation')
model=do_MULTINOMIAL(features_train,labels_train,features_test,labels_test)
test_pred = model.predict_proba(features_test)

print "Lematizador sin Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('lemmatisation',sw=False)
model=do_MULTINOMIAL(features_train,labels_train,features_test,labels_test)

print "Stemming con Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('stemming')
model=do_MULTINOMIAL(features_train,labels_train,features_test,labels_test)

print "Stemming sin Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('stemming',sw=False)
model=do_MULTINOMIAL(features_train,labels_train,features_test,labels_test)

print "Conjunto aleatorio de prueba"
spl = random.sample(xrange(len(test_pred)), 5)
for text, sentiment in zip(test_df.Text[spl], test_pred[spl]):
    print sentiment, text

BI Multinomial
Lematizador con Stopwords
Training Accuracy MULTINOMIAL: 0.959482
Test Accuracy MULTINOMIAL: 0.740782
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.75      0.73      0.74      1803
          -       0.73      0.75      0.74      1751

avg / total       0.74      0.74      0.74      3554

Lematizador sin Stopwords
Training Accuracy MULTINOMIAL: 0.955543
Test Accuracy MULTINOMIAL: 0.747537
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.75      0.75      0.75      1803
          -       0.74      0.74      0.74      1751

avg / total       0.75      0.75      0.75      3554

Stemming con Stopwords
Training Accuracy MULTINOMIAL: 0.942319
Test Accuracy MULTINOMIAL: 0.749789
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.75      0.75      1803
          -       0.74      0.7

No hay mucha diferencia entre multinomial y el método anteriormente usado. Se repite que el mejor comportamiento lo lematización sin stop words. La accurazy en este caso es levemente mejor. Aún así, las métricas de precision, recall, f1-score no cambian mucho.

Entre bayes solo y multinomial la diferencia es que, para Naive Bayes se necesita estimar la distribución de datos y clases. Para el caso de Multinomial simplemente se asume una distribución multinomial para todos los pares de datos. En casos de conteo de palabras es mejor el multinomial.

### **(h) Regresión logistica regularizada **

El siguiente método a utilizar es Regresión logística regularizada. El penalty escogido es la noma $l_2$. Este penalizador se acompaña de un parámetro C que indica el nivel de penalización que se aplicará. Para valores menores de C la penalización es mayor. Con esto se facilita la obtención de características representativas del modelo.

Para valores de C grandes es clara la presencia de overfitting. Cuando toma valores más pequeños se regulariza más por lo que se obtiene un mejor modelo.

In [ ]:
def do_LOGIT(x,y,xt,yt):
    start_t = time.time()
    Cs = [0.01,0.1,1]#,10,100,1000]
    for C in Cs:
        print "Usando C= %f"%C
        model = LogisticRegression(penalty='l2',C=C)
        model = model.fit(x, y)
        score_the_model(model,x,y,xt,yt,"LOGISTIC")
    return model

logistic = []
print "Regresión Logística regularizada"
print "Lematizador con Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('lemmatisation')
model = do_LOGIT(features_train,labels_train,features_test,labels_test)
test_pred = model.predict_proba(features_test)

print "Lematizador sin Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('lemmatisation',sw=False)
model=do_LOGIT(features_train,labels_train,features_test,labels_test)

print "Stemming con StopWords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('stemming')
model=do_LOGIT(features_train,labels_train,features_test,labels_test)

print "Stemming sin StopWords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('stemming',sw=False)
model=do_LOGIT(features_train,labels_train,features_test,labels_test)

print "Conjunto aleatorio de prueba"
spl = random.sample(xrange(len(test_pred)), 5)
for text, sentiment in zip(test_df.Text[spl], test_pred[spl]):
    print sentiment, text

Regresión Logística regularizada
Lematizador con Stopwords
Usando C= 0.010000
Training Accuracy LOGISTIC: 0.784468
Test Accuracy LOGISTIC: 0.678863
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.67      0.73      0.70      1803
          -       0.69      0.63      0.66      1751

avg / total       0.68      0.68      0.68      3554

Usando C= 0.100000
Training Accuracy LOGISTIC: 0.892234
Test Accuracy LOGISTIC: 0.719111
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.72      0.72      0.72      1803
          -       0.72      0.71      0.71      1751

avg / total       0.72      0.72      0.72      3554

Usando C= 1.000000
Training Accuracy LOGISTIC: 0.989589
Test Accuracy LOGISTIC: 0.721362
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.73      0.72      0.72      1803
          -       0.72 

Se decidió eliminar los C mayores a 0.1 por tener accurracy 1 en el training set y un menor valor en el test. Por lo tanto se ajustaron mucho a los datos. Se decidió agregar C=1 para tener un valor de una magnitud mayor.

Para regresión logística Stemming con el logra mejor accuracy en el training set y test set, a diferencia de los métodos anteriores. 

### **(i) SVM **

El siguiente paso es construir una función para SVM o Maquinas de vectores de soporte. Al igual de regresión logistica se utiliza un penalty con el parámetro C. Si se repite el mismo comportamiento anterior, para valores de C grande se producirá un ajuste a los datos de entrenamiento.

In [ ]:
def do_SVM(x,y,xt,yt):
    Cs = [0.01,0.1,10,100,1000]
    for C in Cs:
        print "El valor de C que se esta probando: %f"%C
        model = LinearSVC(C=C)
        model = model.fit(x, y)
        score_the_model(model,x,y,xt,yt,"SVM")
    return model

svm = []
print "Maquina de vectores de soporte"
print "Lematizador con Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('lemmatisation')        
model = do_SVM(features_train,labels_train,features_test,labels_test)
test_pred = model.predict(features_test)

print "Lematizador sin Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('lemmatisation',sw=False)
model=do_SVM(features_train,labels_train,features_test,labels_test)

print "Stemming con Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('stemming')
model=do_SVM(features_train,labels_train,features_test,labels_test)

print "Stemming sin Stopwords"
features_train, features_test, labels_train, labels_test, vocab, dist_train = vector_rep('stemming',sw=False)
model=do_SVM(features_train,labels_train,features_test,labels_test)

print "Conjunto aleatorio de prueba"
spl = random.sample(xrange(len(test_pred)), 5)
for text, sentiment in zip(test_df.Text[spl], test_pred[spl]):
    print sentiment, text

Maquina de vectores de soporte
Lematizador con Stopwords


Como se predijo antes, se repitió el comportamiento para penalizaciones bajas. El modelo tendió a ajustarse a los datos de entrenamiento. Observando las accuracy para C igual a 0.01 y 0.1 se obtiene una alta accuracy para el entrenamiento y test, siendo mejor para un valor de C=0.1. El mejor modelo es obtenido utilizando stemming por tener un mayor poder predictivo con el data de prueba. Aún así, los demás métodos también alcanzan una accuracy cercana.

### **(j) Comparación de modelos **

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors

accuracy_train = []
accuracy_test = []
precision = []
recall = []
fscore = []
print BNB
'''
#accuracy_train.extend((BNB[0],multinomial[0],logistic[0],svm[0]))
accuracy_test.extend((BNB[1],multinomial[1],logistic[1],svm[1]))
precision.extend((BNB[2],multinomial[2],logistic[2],svm[2]))
recall.extend((BNB[3],multinomial[3],logistic[3],svm[3]))
fscore.extend((BNB[4],multinomial[4],logistic[4],svm[4]))

fig, ax = plt.subplots()
index = np.arange(4)
bar_width = 0.15
opacity = 0.9
 
rects1 = plt.bar(index, accuracy_train, bar_width,alpha=opacity,color='b',label='Accuracy Train')
rects2 = plt.bar(index + bar_width, accuracy_test, bar_width,alpha=opacity,color='g',label='Accuracy Test')
rects3 = plt.bar(index + 2*bar_width, precision, bar_width,alpha=opacity,color='r',label='Precision')
rects4 = plt.bar(index + 3*bar_width, recall, bar_width,alpha=opacity,color='c',label='Recall')
rects5 = plt.bar(index + 4*bar_width, fscore, bar_width,alpha=opacity,color='m',label='F1-Score')
plt.xlabel('Metodos de Clasificacion')
plt.ylabel('Scores')
plt.title('Metricas')
plt.xticks(index + bar_width, ('BernoulliNB', 'MultinomialNB', 'LogisticRegression', 'LinearSVC'))
plt.legend()
plt.show()'''